Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/explain-model/azure-integration/scoring-time/train-explain-model-on-amlcompute-and-deploy.png)

# Train and explain models remotely via Azure Machine Learning Compute and deploy model and scoring explainer


_**This notebook illustrates how to use the Azure Machine Learning Interpretability SDK to train and explain a classification model remotely on an Azure Machine Leanrning Compute Target (AMLCompute), and use Azure Container Instances (ACI) for deploying your model and its corresponding scoring explainer as a web service.**_

Problem: IBM employee attrition classification with scikit-learn (train a model and run an explainer remotely via AMLCompute, and deploy model and its corresponding explainer.)

---

## Table of Contents

1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Run model explainer locally at training time](#Explain)
    1. Apply feature transformations
    1. Train a binary classification model
    1. Explain the model on raw features
        1. Generate global explanations
        1. Generate local explanations
1. [Visualize results](#Visualize)
1. [Deploy model and scoring explainer](#Deploy)
1. [Next steps](#Next)

## Introduction

This notebook showcases how to train and explain a classification model remotely via Azure Machine Learning Compute (AMLCompute), download the calculated explanations locally for visualization and inspection, and deploy the final model and its corresponding explainer to Azure Container Instances (ACI).
It demonstrates the API calls that you need to make to submit a run for training and explaining a model to AMLCompute, download the compute explanations remotely, and visualizing the global and local explanations via a visualization dashboard that provides an interactive way of discovering patterns in model predictions and downloaded explanations, and using Azure Machine Learning MLOps capabilities to deploy your model and its corresponding explainer.

We will showcase one of the tabular data explainers: TabularExplainer (SHAP) and follow these steps:
1.	Develop a machine learning script in Python which involves the training script and the explanation script.
2.	Create and configure a compute target.
3.	Submit the scripts to the configured compute target to run in that environment. During training, the scripts can read from or write to datastore. And the records of execution (e.g., model, metrics, prediction explanations) are saved as runs in the workspace and grouped under experiments.
4.	Query the experiment for logged metrics and explanations from the current and past runs. Use the interpretability toolkitâ€™s visualization dashboard to visualize predictions and their explanation. If the metrics and explanations don't indicate a desired outcome, loop back to step 1 and iterate on your scripts.
5.	After a satisfactory run is found, create a scoring explainer and register the persisted model and its corresponding explainer in the model registry.
6.	Develop a scoring script.
7.	Create an image and register it in the image registry.
8.	Deploy the image as a web service in Azure.

| ![azure-machine-learning-cycle](./img/azure-machine-learning-cycle.PNG) |
|:--:|

## Setup
Make sure you go through the [configuration notebook](../../../../configuration.ipynb) first if you haven't.

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize a Workspace

Initialize a workspace object from persisted configuration

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

## Explain

Create An Experiment: **Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [ ]:
from azureml.core import Experiment
experiment_name = 'explainer-remote-run-on-amlcompute'
experiment = Experiment(workspace=ws, name=experiment_name)

## Introduction to AmlCompute

Azure Machine Learning Compute is managed compute infrastructure that allows the user to easily create single to multi-node compute of the appropriate VM Family. It is created **within your workspace region** and is a resource that can be used by other users in your workspace. It autoscales by default to the max_nodes, when a job is submitted, and executes in a containerized environment packaging the dependencies as specified by the user. 

Since it is managed compute, job scheduling and cluster management are handled internally by Azure Machine Learning service. 

For more information on Azure Machine Learning Compute, please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)

If you are an existing BatchAI customer who is migrating to Azure Machine Learning, please read [this article](https://aka.ms/batchai-retirement)

**Note**: As with other Azure services, there are limits on certain resources (for eg. AmlCompute quota) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.


The training script `run_explainer.py` is already created for you. Let's have a look.

## Submit an AmlCompute run in a few different ways

First lets check which VM families are available in your region. Azure is a regional service and some specialized SKUs (especially GPUs) are only available in certain regions. Since AmlCompute is created in the region of your workspace, we will use the supported_vms () function to see if the VM family we want to use ('STANDARD_D2_V2') is supported.

You can also pass a different region to check availability and then re-create your workspace in that region through the [configuration notebook](../../../configuration.ipynb)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

AmlCompute.supported_vmsizes(workspace=ws)
# AmlCompute.supported_vmsizes(workspace=ws, location='southcentralus')

### Create project directory

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on

In [ ]:
import os
import shutil

project_folder = './explainer-remote-run-on-amlcompute'
os.makedirs(project_folder, exist_ok=True)
shutil.copy('train_explain.py', project_folder)

### Provision as a run based compute target

You can provision AmlCompute as a compute target at run-time. In this case, the compute is auto-created for your run, scales up to max_nodes that you specify, and then **deleted automatically** after the run completes.

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# create a new runconfig object
run_config = RunConfiguration()

# signal that you want to use AmlCompute to execute script.
run_config.target = "amlcompute"

# AmlCompute will be created in the same region as workspace
# Set vm size for AmlCompute
run_config.amlcompute.vm_size = 'STANDARD_D2_V2'

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# auto-prepare the Docker image when used for execution (if it is not already prepared)
run_config.auto_prepare_environment = True

azureml_pip_packages = [
    'azureml-defaults', 'azureml-contrib-interpret', 'azureml-core', 'azureml-telemetry',
    'azureml-interpret', 'azureml-dataprep'
]
 


# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'],
                                pip_packages=['sklearn_pandas', 'pyyaml'] + azureml_pip_packages,
                                pin_sdk_version=False)
# Now submit a run on AmlCompute
from azureml.core.script_run_config import ScriptRunConfig

script_run_config = ScriptRunConfig(source_directory=project_folder,
                                    script='train_explain.py',
                                    run_config=run_config)

run = experiment.submit(script_run_config)

# Show run details
run

Note: if you need to cancel a run, you can follow [these instructions](https://aka.ms/aml-docs-cancel-run).

In [ ]:
%%time
# Shows output of the run on stdout.
run.wait_for_completion(show_output=True)

In [ ]:
# Delete () is used to deprovision and delete the AmlCompute target. Useful if you want to re-use the compute name 
# 'cpucluster' in this case but use a different VM family for instance.

# cpu_cluster.delete()

## Download Model Explanation, Model, and Data

In [ ]:
# retrieve model for visualization and deployment
from azureml.core.model import Model
from sklearn.externals import joblib
original_model = Model(ws, 'amlcompute_deploy_model')
model_path = original_model.download(exist_ok=True)
original_svm_model = joblib.load(model_path)

In [ ]:
# retrieve global explanation for visualization
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

# get model explanation data
client = ExplanationClient.from_run(run)
global_explanation = client.download_model_explanation()

In [ ]:
# retrieve x_test for visualization
from sklearn.externals import joblib
x_test_path = './x_test.pkl'
run.download_file('x_test_ibm.pkl', output_file_path=x_test_path)
x_test = joblib.load(x_test_path)

## Visualize
Visualize the explanations

In [ ]:
from azureml.contrib.interpret.visualize import ExplanationDashboard

In [ ]:
ExplanationDashboard(global_explanation, original_svm_model, x_test)

## Deploy
Deploy Model and ScoringExplainer

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

# WARNING: to install this, g++ needs to be available on the Docker image and is not by default (look at the next cell)
azureml_pip_packages = [
    'azureml-defaults', 'azureml-contrib-interpret', 'azureml-core', 'azureml-telemetry',
    'azureml-interpret'
]
 

# specify CondaDependencies obj
myenv = CondaDependencies.create(conda_packages=['scikit-learn', 'pandas'],
                                 pip_packages=['sklearn-pandas', 'pyyaml'] + azureml_pip_packages,
                                 pin_sdk_version=False)

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

with open("myenv.yml","r") as f:
    print(f.read())

In [ ]:
%%writefile dockerfile
RUN apt-get update && apt-get install -y g++ 

In [ ]:
# retrieve scoring explainer for deployment
scoring_explainer_model = Model(ws, 'IBM_attrition_explainer')

In [ ]:
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "IBM_Attrition",  
                                                     "method" : "local_explanation"}, 
                                               description='Get local explanations for IBM Employee Attrition data')

inference_config = InferenceConfig(runtime= "python", 
                                   entry_script="score_remote_explain.py",
                                   conda_file="myenv.yml",
                                   extra_docker_file_steps="dockerfile")

# Use configs and models generated above
service = Model.deploy(ws, 'model-scoring-service', [scoring_explainer_model, original_model], inference_config, aciconfig)
service.wait_for_deployment(show_output=True)

In [ ]:
import requests

# create data to test service with
examples = x_test[:4]
input_data = examples.to_json()

headers = {'Content-Type':'application/json'}

# send request to service
resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
# can covert back to Python objects from json string if desired
print("prediction:", resp.text)

In [ ]:
service.delete()

## Next
Learn about other use cases of the explain package on a:
1. [Training time: regression problem](../../tabular-data/explain-binary-classification-local.ipynb)       
1. [Training time: binary classification problem](../../tabular-data/explain-binary-classification-local.ipynb)
1. [Training time: multiclass classification problem](../../tabular-data/explain-multiclass-classification-local.ipynb)
1. Explain models with engineered features:
    1. [Simple feature transformations](../../tabular-data/simple-feature-transformations-explain-local.ipynb)
    1. [Advanced feature transformations](../../tabular-data/advanced-feature-transformations-explain-local.ipynb)
1. [Save model explanations via Azure Machine Learning Run History](../run-history/save-retrieve-explanations-run-history.ipynb)
1. [Run explainers remotely on Azure Machine Learning Compute (AMLCompute)](../remote-explanation/explain-model-on-amlcompute.ipynb)
1. [Inferencing time: deploy a locally-trained model and explainer](./train-explain-model-locally-and-deploy.ipynb)
   